# Setup
This IPython notebook will walk through the steps of characterizing iModulons through the semi-automated tools in PyModulon. You will need:

* M and A matrices
* Expression data (e.g. `log_tpm.csv`)
* Gene table and KEGG/GO annotations (Generated in `gene_annotation.ipynb`)
* Sample table, with a column for `project` and `condition`
* TRN file

Optional:
* iModulon table (if you already have some characterized iModulons)

In [1]:
from pymodulon.core import IcaData
from pymodulon.plotting import *
import os
from os import path
import pandas as pd
import re
from Bio.KEGG import REST
from tqdm.notebook import tqdm
from pymodulon.util import explained_variance
from pymodulon.io import *


In [2]:
# Enter the location of your data here
data_dir = path.join('..','data')

# GO and KEGG annotations are in the 'external' folder
external_data = path.join('..','data')

## Check your sample table (i.e. metadata file)
Your metadata file will probably have a lot of columns, most of which you may not care about. Feel free to save a secondary copy of your metadata file with only columns that seem relevant to you. The two most important columns are:
1. `project`
2. `condition`

Make sure that these columns exist in your metadata file

In [3]:
#df_metadata = pd.read_csv(os.path.join('..','data','metadata.tsv',index_col=0,sep='\t'))
df_metadata = pd.read_csv(os.path.join('..','data','sample.csv'),index_col=0,sep=',')
df_metadata[['project','condition']].head()

,project,condition
ERR1799216,sigD,WT
ERR1799217,sigD,DEL-sigD
ERR1799218,sigD,pVWEx1-sigD
ERR2401409,altering-oxygen,JL-3h-2-Stuttgart
ERR2401410,altering-oxygen,JL-5h-2-Stuttgart


In [4]:
print(df_metadata.project.notnull().all())
print(df_metadata.condition.notnull().all())

False
False


## Check your TRN

Each row of the TRN file represents a regulatory interaction.  
**Your TRN file must have the following columns:**
1. `regulator` - Name of regulator (`/` or `+` characters will be converted to `;`)
1. `gene_id` - Locus tag of gene being regulated

The following columns are optional, but are helpful to have:
1. `regulator_id` - Locus tag of regulator
1. `gene_name` - Name of gene (can automatically update this using `name2num`)
1. `direction` - Direction of regulation ('+' for activation, '-' for repression, '?' or NaN for unknown)
1. `evidence` - Evidence of regulation (e.g. ChIP-exo, qRT-PCR, SELEX, Motif search)
1. `PMID` - Reference for regulation

You may add any other columns that could help you. TRNs may be saved as either CSV or TSV files. See below for an example:

In [5]:
df_trn = pd.read_csv(os.path.join('..','data','TRN.csv'))
df_trn.head()

,regulator,gene_id,effect
0,Cobalamin,WA5_RS02685,NaN
1,Cobalamin,WA5_RS02690,NaN
2,Cobalamin,WA5_RS02695,NaN
3,Cobalamin,WA5_RS02700,NaN
4,Cobalamin,WA5_RS02705,NaN


The `regulator` and `gene_id` must be filled in for each row

In [6]:
print(df_trn.regulator.notnull().all())
print(df_trn.gene_id.notnull().all())

True
True


## Load the data
You're now ready to load your IcaData object!

In [7]:
#df_mapping = pd.read_excel(os.path.join('..','data','MM_2ldfeeem.emapper.annotations.xlsx'))
df_mapping = pd.read_excel(os.path.join('..','data','eggNOG_annotations.xlsx'))
df_trn['regulator_locus_tag']=''
df_trn['gene_id_locus_tag']=''
dict = {}
for row in df_mapping.index:
    maps = {df_mapping.loc[row]['seed_ortholog'].split(".")[1]:df_mapping.loc[row]['query']}
    dict.update(maps)
for row in df_trn.index:
    df_trn.loc[row]['regulator_locus_tag'] = dict.get(df_trn.loc[row]['regulator'])
    df_trn.loc[row]['gene_id_locus_tag'] = dict.get(df_trn.loc[row]['gene_id'])
df_trn.to_excel(os.path.join('..','data','trn_test.xlsx'))

In [8]:
#data = pd.read_excel(os.path.join('..','data','TRN.xlsx',index_col=0))
# data = pd.read_excel(os.path.join('..','data','trn_test.xlsx',index_col=0))
# data.to_csv(os.path.join('..','data','trn_test.xlsx',encoding='utf-8'))
data = pd.read_excel(os.path.join('..','data','trn_test.xlsx'),index_col=0)
data.to_csv(os.path.join('..','data','trn_test.csv'),encoding='utf-8')

Use the following code to check if the ICA data meets the requirements.

In [9]:

M = pd.read_csv(os.path.join('..','data','M.csv'))
A = pd.read_csv(os.path.join('..','data','A.csv'))
X = pd.read_csv(os.path.join('..','data','log_tpm.csv'))
print(X.columns.tolist())
print(A.columns.tolist())

if X.columns.tolist() != A.columns.tolist():
    # Find the sample names that are out of order
    for i, (x_sample, a_sample) in enumerate(zip(X.columns, A.columns)):
        if x_sample != a_sample:
            print(f"Order mismatch: Sample '{x_sample}' in X matrix and sample '{a_sample}' in A matrix at position {i}")
else:
    print("The order of sample names is consistent, no need to fix")



['Geneid', 'ERR1799216', 'ERR1799217', 'ERR1799218', 'ERR2401409', 'ERR2401410', 'ERR2401411', 'ERR2401412', 'ERR2401413', 'ERR2401414', 'ERR2601590', 'ERR2601591', 'ERR2601592', 'ERR2601593', 'ERR2601595', 'ERR2601596', 'ERR2601597', 'ERR2601598', 'ERR2601599', 'ERR2601600', 'ERR2601601', 'ERR2601602', 'ERR2601603', 'ERR2601604', 'ERR2601605', 'ERR2601606', 'ERR2601607', 'ERR2601608', 'ERR2601609', 'ERR2601610', 'ERR2601611', 'ERR2601612', 'ERR2601613', 'ERR2601614', 'ERR2601615', 'ERR2601616', 'ERR2601617', 'ERR2601618', 'ERR2601619', 'ERR2601620', 'ERR2601621', 'ERR2601622', 'ERR2601623', 'ERR2601624', 'ERR2601625', 'ERR2601626', 'ERR2601627', 'ERR2601628', 'ERR2601629', 'ERR2601630', 'ERR2601631', 'ERR2601632', 'ERR2601633', 'ERR2868910', 'ERR2868911', 'ERR2936684', 'ERR2936685', 'ERR2936686', 'ERR2936687', 'ERR2936688', 'ERR2936689', 'ERR2936690', 'ERR2936691', 'ERR2936692', 'ERR2936693', 'ERR2936694', 'ERR2936695', 'ERR2936696', 'ERR2936697', 'ERR2936698', 'ERR2936699', 'ERR29367

File "/tmp/ipykernel_1020952/226201940.py", line 10
    optimize_cutoff=True)
                  ^
SyntaxError: invalid syntax

In [10]:
ica_data = IcaData(M = path.join(data_dir,'M.csv'),
                   A = path.join(data_dir,'A.csv'),
                   #X = path.join(data_dir,'log_tpm_test.csv'),
                   X = path.join(data_dir,'log_tpm.csv'),
                   gene_table = path.join(data_dir,'gene_info.csv'),
                   #sample_table = path.join(data_dir,'metadata_test.tsv'),
                   sample_table = df_metadata,
                   trn = path.join(data_dir,'trn_test.csv'),
                   #trn = path.join(data_dir,'trn_atlas.csv'),
                   optimize_cutoff=True)

If you don't have a TRN (or have a very minimal TRN), use `threshold_method = 'kmeans'`

In [11]:
ica_data = IcaData(M = path.join(data_dir,'M.csv'),
                   A = path.join(data_dir,'A.csv'),
                   X = path.join(data_dir,'log_tpm.csv'),
                   gene_table = path.join(data_dir,'gene_info.csv'),
                   #sample_table = path.join(data_dir,'metadata_test.tsv'),
                   sample_table = df_metadata,
                   trn = path.join(data_dir,'trn_test.csv'),
                   #trn = path.join(data_dir,'trn_atlas.csv'),
                   optimize_cutoff=True)
#                    threshold_method = 'kmeans'

# Regulatory iModulons
Use `compute_trn_enrichment` to automatically check for Regulatory iModulons. The more complete your TRN, the more regulatory iModulons you'll find.

In [12]:
ica_data.compute_trn_enrichment()


,imodulon,regulator,pvalue,qvalue,precision,recall,f1score,TP,regulon_size,imodulon_size,n_regs
0,3,WA5_RS03145,2.913623e-11,7.284058e-10,0.054054,1.000000,0.102564,8.0,8.0,148.0,1.0
1,3,WA5_RS14715,1.126364e-07,1.407955e-06,0.087838,0.288889,0.134715,13.0,45.0,148.0,1.0
2,5,WA5_RS11920,5.588787e-10,3.353272e-09,0.444444,0.666667,0.533333,4.0,6.0,9.0,1.0
3,5,WA5_RS11905,1.829565e-08,5.488695e-08,0.444444,0.333333,0.380952,4.0,12.0,9.0,1.0
4,5,WA5_RS11985,1.553926e-06,3.107852e-06,0.333333,0.333333,0.333333,3.0,9.0,9.0,1.0
5,7,WA5_RS00825,4.502822e-17,9.005644e-17,1.000000,0.411765,0.583333,7.0,17.0,7.0,1.0
6,7,WA5_RS01525,1.316009e-06,1.316009e-06,0.857143,0.025862,0.050209,6.0,232.0,7.0,1.0
7,9,WA5_RS07815,5.184490e-11,3.110694e-10,0.800000,0.571429,0.666667,4.0,7.0,5.0,1.0
8,9,WA5_RS15475,2.689453e-09,8.068358e-09,0.800000,0.250000,0.380952,4.0,16.0,5.0,1.0
9,9,WA5_RS09590,1.079216e-08,2.158433e-08,0.800000,0.181818,0.296296,4.0,22.0,5.0,1.0


You can also search for AND/OR combinations of regulators using the `max_regs` argument.

Regulator enrichments can be directly saved to the `imodulon_table` using the `save` argument. This saves the enrichment with the lowest q-value to the table.

In [13]:
# First search for regulator enrichments with 2 regulators
ica_data.compute_trn_enrichment(max_regs=2,save=True)

# Next, search for regulator enrichments with just one regulator. This will supercede the 2 regulator enrichments.
ica_data.compute_trn_enrichment(max_regs=1,save=True)

,imodulon,regulator,pvalue,qvalue,precision,recall,f1score,TP,regulon_size,imodulon_size,n_regs
0,3,WA5_RS03145,2.913623e-11,7.284058e-10,0.054054,1.000000,0.102564,8.0,8.0,148.0,1.0
1,3,WA5_RS14715,1.126364e-07,1.407955e-06,0.087838,0.288889,0.134715,13.0,45.0,148.0,1.0
2,5,WA5_RS11920,5.588787e-10,3.353272e-09,0.444444,0.666667,0.533333,4.0,6.0,9.0,1.0
3,5,WA5_RS11905,1.829565e-08,5.488695e-08,0.444444,0.333333,0.380952,4.0,12.0,9.0,1.0
4,5,WA5_RS11985,1.553926e-06,3.107852e-06,0.333333,0.333333,0.333333,3.0,9.0,9.0,1.0
5,7,WA5_RS00825,4.502822e-17,9.005644e-17,1.000000,0.411765,0.583333,7.0,17.0,7.0,1.0
6,7,WA5_RS01525,1.316009e-06,1.316009e-06,0.857143,0.025862,0.050209,6.0,232.0,7.0,1.0
7,9,WA5_RS07815,5.184490e-11,3.110694e-10,0.800000,0.571429,0.666667,4.0,7.0,5.0,1.0
8,9,WA5_RS15475,2.689453e-09,8.068358e-09,0.800000,0.250000,0.380952,4.0,16.0,5.0,1.0
9,9,WA5_RS09590,1.079216e-08,2.158433e-08,0.800000,0.181818,0.296296,4.0,22.0,5.0,1.0


The list of regulatory iModulons are shown below

In [14]:
regulatory_imodulons = ica_data.imodulon_table[ica_data.imodulon_table.regulator.notnull()]
print(len(ica_data.imodulon_table),'Total iModulons')
print(len(regulatory_imodulons),'Regulatory iModulons')
regulatory_imodulons

87 Total iModulons
19 Regulatory iModulons


,regulator,pvalue,qvalue,precision,recall,f1score,TP,regulon_size,imodulon_size,n_regs
3,WA5_RS03145,2.913623e-11,7.284058e-10,0.054054,1.000000,0.102564,8.0,8.0,148.0,1.0
5,WA5_RS11920,5.588787e-10,3.353272e-09,0.444444,0.666667,0.533333,4.0,6.0,9.0,1.0
7,WA5_RS00825,4.502822e-17,9.005644e-17,1.000000,0.411765,0.583333,7.0,17.0,7.0,1.0
9,WA5_RS07815,5.184490e-11,3.110694e-10,0.800000,0.571429,0.666667,4.0,7.0,5.0,1.0
11,WA5_RS11355,5.711011e-11,1.713303e-10,0.454545,0.500000,0.476190,5.0,10.0,11.0,1.0
20,TPP,4.493335e-11,1.348000e-10,0.315789,0.461538,0.375000,6.0,13.0,19.0,1.0
30,WA5_RS04910,6.871305e-07,5.497044e-06,0.333333,0.200000,0.250000,4.0,20.0,12.0,1.0
31,WA5_RS02795,9.984621e-18,2.995386e-17,0.700000,0.777778,0.736842,7.0,9.0,10.0,1.0
33,WA5_RS13915,8.578858e-07,6.005201e-06,0.157895,0.750000,0.260870,3.0,4.0,19.0,1.0
35,WA5_RS05330,8.884442e-09,2.665333e-08,0.600000,0.750000,0.666667,3.0,4.0,5.0,1.0


You can rename iModulons in this jupyter notebook, or you can save the iModulon table as a CSV and edit it in Excel.

If two iModulons have the same regulator (e.g. 'Reg'), they will be named 'Reg-1' and 'Reg-2'

In [15]:
ica_data.rename_imodulons(regulatory_imodulons.regulator.to_dict())
ica_data.imodulon_table.head()

,regulator,pvalue,qvalue,precision,recall,f1score,TP,regulon_size,imodulon_size,n_regs
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WA5_RS03145,WA5_RS03145,2.913623e-11,7.284058e-10,0.054054,1.0,0.102564,8.0,8.0,148.0,1.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
regulatory_imodulons = ica_data.imodulon_table[ica_data.imodulon_table.regulator.notnull()]

# Functional iModulons

GO annotations and KEGG pathways/modules were generated in the 1_create_the_gene_table.ipynb notebook. Enrichments will be calculated in this notebook, and further curated in the 3_manual_iModulon_curation notebook.

## GO Enrichments

First load the Gene Ontology annotations

In [17]:
DF_GO = pd.read_csv(os.path.join('..','data','GO_annotations_curated.csv'),index_col=0)
DF_GO.head()

,gene_id,gene_name,gene_ontology
1,WA5_RS10870,pimB,phosphatidylinositol alpha-mannosyltransferase...
2,WA5_RS10870,pimB,"initiation-specific glycolipid 1,6-alpha-manno..."
3,WA5_RS10870,pimB,glycolipid biosynthetic process
19,WA5_RS13360,radA,SOS response
20,WA5_RS14450,glpK,glycerol kinase activity


In [18]:
DF_GO_enrich = ica_data.compute_annotation_enrichment(DF_GO,'gene_ontology')

In [19]:
DF_GO_enrich

,imodulon,gene_ontology,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size
0,6,L-histidine biosynthetic process,0.005332,0.055981,0.062500,1.0,0.117647,1.0,1.0,16.0
1,6,histidinol-phosphatase activity,0.005332,0.055981,0.062500,1.0,0.117647,1.0,1.0,16.0
2,WA5_RS00045,"fructose 1,6-bisphosphate 1-phosphatase activity",0.002999,0.015745,0.111111,1.0,0.200000,1.0,1.0,9.0
3,WA5_RS00045,"fructose 1,6-bisphosphate metabolic process",0.002999,0.015745,0.111111,1.0,0.200000,1.0,1.0,9.0
4,WA5_RS00045,gluconeogenesis,0.002999,0.015745,0.111111,1.0,0.200000,1.0,1.0,9.0
5,WA5_RS00045,manganese ion binding,0.002999,0.015745,0.111111,1.0,0.200000,1.0,1.0,9.0
6,WA5_RS00045,magnesium ion binding,0.005990,0.025158,0.111111,0.5,0.181818,1.0,2.0,9.0


## KEGG Enrichments

### Load KEGG mapping
The `kegg_mapping.csv` file contains KEGG orthologies, pathways, modules, and reactions. Only pathways and modules are relevant to iModulon characterization.

In [20]:
DF_KEGG = pd.read_csv(os.path.join('..','data','kegg_mapping.csv'),index_col=0)
print(DF_KEGG.database.unique())
DF_KEGG.head()

['KEGG_Pathway' 'KEGG_Module' 'KEGG_Reaction']


,gene_id,database,kegg_id
1878,WA5_RS00005,KEGG_Pathway,map02020
1879,WA5_RS00005,KEGG_Pathway,map04112
1886,WA5_RS00010,KEGG_Pathway,map00230
1887,WA5_RS00010,KEGG_Pathway,map00240
1888,WA5_RS00010,KEGG_Pathway,map01100


In [21]:
kegg_pathways = DF_KEGG[DF_KEGG.database == 'KEGG_Pathway']
kegg_modules = DF_KEGG[DF_KEGG.database == 'KEGG_Module']

In [22]:
print(kegg_pathways)

          gene_id      database   kegg_id
1878  WA5_RS00005  KEGG_Pathway  map02020
1879  WA5_RS00005  KEGG_Pathway  map04112
1886  WA5_RS00010  KEGG_Pathway  map00230
1887  WA5_RS00010  KEGG_Pathway  map00240
1888  WA5_RS00010  KEGG_Pathway  map01100
...           ...           ...       ...
8725  WA5_RS15490  KEGG_Pathway  map01503
8729  WA5_RS15515  KEGG_Pathway  map02024
8730  WA5_RS15515  KEGG_Pathway  map03060
8731  WA5_RS15515  KEGG_Pathway  map03070
8733  WA5_RS15525  KEGG_Pathway  map03010

[3429 rows x 3 columns]


### Perform enrichment
Uses the `compute_annotation_enrichment` function

In [23]:
DF_pathway_enrich = ica_data.compute_annotation_enrichment(kegg_pathways,'kegg_id')
DF_module_enrich = ica_data.compute_annotation_enrichment(kegg_modules,'kegg_id')

In [24]:
DF_pathway_enrich

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size
0,2,map00860,1.989111e-04,4.495392e-02,0.666667,0.080000,0.142857,2.0,25.0,3.0
1,WA5_RS03145,map00364,5.626123e-05,1.271504e-02,0.033784,0.500000,0.063291,5.0,10.0,148.0
2,WA5_RS03145,map01220,3.038931e-04,3.433992e-02,0.047297,0.250000,0.079545,7.0,28.0,148.0
3,WA5_RS03145,map00362,1.625536e-03,7.347425e-02,0.040541,0.222222,0.068571,6.0,27.0,148.0
4,WA5_RS03145,map00622,1.092728e-03,7.347425e-02,0.020270,0.600000,0.039216,3.0,5.0,148.0
5,WA5_RS03145,map00920,1.386210e-03,7.347425e-02,0.033784,0.277778,0.060241,5.0,18.0,148.0
6,WA5_RS11920,map00362,7.315561e-14,1.101563e-11,0.777778,0.259259,0.388889,7.0,27.0,9.0
7,WA5_RS11920,map01220,9.748347e-14,1.101563e-11,0.777778,0.250000,0.378378,7.0,28.0,9.0
8,WA5_RS11920,map01120,1.141398e-07,8.598532e-06,0.777778,0.037037,0.070707,7.0,189.0,9.0
9,WA5_RS11920,map00364,2.216559e-06,1.252356e-04,0.333333,0.300000,0.315789,3.0,10.0,9.0


In [25]:
DF_module_enrich

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size
0,WA5_RS03145,M00176,5.689899e-06,1.325746e-03,0.027027,1.000000,0.052632,4.0,4.0,148.0
1,WA5_RS03145,M00207,2.929142e-05,3.412450e-03,0.033784,0.555556,0.063694,5.0,9.0,148.0
2,WA5_RS03145,M00238,1.176436e-04,9.136984e-03,0.020270,1.000000,0.039735,3.0,3.0,148.0
3,WA5_RS03145,M00551,4.535046e-04,2.641664e-02,0.020270,0.750000,0.039474,3.0,4.0,148.0
4,WA5_RS11920,M00568,6.494186e-07,1.513145e-04,0.333333,0.428571,0.375000,3.0,7.0,9.0
5,WA5_RS11355,M00244,7.289387e-07,1.698427e-04,0.272727,0.500000,0.352941,3.0,6.0,11.0
6,12,M00479,6.664445e-07,1.552816e-04,0.666667,1.000000,0.800000,2.0,2.0,3.0
7,19,M00298,3.021215e-05,7.039431e-03,0.117647,1.000000,0.210526,2.0,2.0,17.0
8,TPP,M00127,7.416723e-06,1.728097e-03,0.157895,0.428571,0.230769,3.0,7.0,19.0
9,23,M00436,4.166867e-04,9.708800e-02,0.055556,0.666667,0.102564,2.0,3.0,36.0


### Convert KEGG IDs to human-readable names

In [26]:
for idx,key in tqdm(DF_pathway_enrich.kegg_id.items(),total=len(DF_pathway_enrich)):
    text = REST.kegg_find('pathway',key).read()
    try:
        name = re.search('\t(.*)\n',text).group(1)
        DF_pathway_enrich.loc[idx,'pathway_name'] = name
    except AttributeError:
        DF_pathway_enrich.loc[idx,'pathway_name'] = None
    
for idx,key in tqdm(DF_module_enrich.kegg_id.items(),total=len(DF_module_enrich)):
    text = REST.kegg_find('module',key).read()
    try:
        name = re.search('\t(.*)\n',text).group(1)
        DF_module_enrich.loc[idx,'module_name'] = name
    except AttributeError:
        DF_module_enrich.loc[idx,'module_name'] = None

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

In [27]:
DF_pathway_enrich

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size,pathway_name
0,2,map00860,1.989111e-04,4.495392e-02,0.666667,0.080000,0.142857,2.0,25.0,3.0,Porphyrin metabolism
1,WA5_RS03145,map00364,5.626123e-05,1.271504e-02,0.033784,0.500000,0.063291,5.0,10.0,148.0,Fluorobenzoate degradation
2,WA5_RS03145,map01220,3.038931e-04,3.433992e-02,0.047297,0.250000,0.079545,7.0,28.0,148.0,Degradation of aromatic compounds
3,WA5_RS03145,map00362,1.625536e-03,7.347425e-02,0.040541,0.222222,0.068571,6.0,27.0,148.0,Benzoate degradation
4,WA5_RS03145,map00622,1.092728e-03,7.347425e-02,0.020270,0.600000,0.039216,3.0,5.0,148.0,Xylene degradation
5,WA5_RS03145,map00920,1.386210e-03,7.347425e-02,0.033784,0.277778,0.060241,5.0,18.0,148.0,Sulfur metabolism
6,WA5_RS11920,map00362,7.315561e-14,1.101563e-11,0.777778,0.259259,0.388889,7.0,27.0,9.0,Benzoate degradation
7,WA5_RS11920,map01220,9.748347e-14,1.101563e-11,0.777778,0.250000,0.378378,7.0,28.0,9.0,Degradation of aromatic compounds
8,WA5_RS11920,map01120,1.141398e-07,8.598532e-06,0.777778,0.037037,0.070707,7.0,189.0,9.0,Microbial metabolism in diverse environments
9,WA5_RS11920,map00364,2.216559e-06,1.252356e-04,0.333333,0.300000,0.315789,3.0,10.0,9.0,Fluorobenzoate degradation


In [28]:
DF_module_enrich

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size,module_name
0,WA5_RS03145,M00176,5.689899e-06,1.325746e-03,0.027027,1.000000,0.052632,4.0,4.0,148.0,"Assimilatory sulfate reduction, sulfate => H2S"
1,WA5_RS03145,M00207,2.929142e-05,3.412450e-03,0.033784,0.555556,0.063694,5.0,9.0,148.0,None
2,WA5_RS03145,M00238,1.176436e-04,9.136984e-03,0.020270,1.000000,0.039735,3.0,3.0,148.0,None
3,WA5_RS03145,M00551,4.535046e-04,2.641664e-02,0.020270,0.750000,0.039474,3.0,4.0,148.0,"Benzoate degradation, benzoate => catechol / m..."
4,WA5_RS11920,M00568,6.494186e-07,1.513145e-04,0.333333,0.428571,0.375000,3.0,7.0,9.0,"Catechol ortho-cleavage, catechol => 3-oxoadipate"
5,WA5_RS11355,M00244,7.289387e-07,1.698427e-04,0.272727,0.500000,0.352941,3.0,6.0,11.0,None
6,12,M00479,6.664445e-07,1.552816e-04,0.666667,1.000000,0.800000,2.0,2.0,3.0,None
7,19,M00298,3.021215e-05,7.039431e-03,0.117647,1.000000,0.210526,2.0,2.0,17.0,None
8,TPP,M00127,7.416723e-06,1.728097e-03,0.157895,0.428571,0.230769,3.0,7.0,19.0,"Thiamine biosynthesis, prokaryotes, AIR (+ DXP..."
9,23,M00436,4.166867e-04,9.708800e-02,0.055556,0.666667,0.102564,2.0,3.0,36.0,None


## Save files

In [29]:
DF_GO_enrich['source'] = 'GO'
# DF_pathway_enrich['source'] = 'KEGG pathways'
# DF_module_enrich['source'] = 'KEGG modules'
# DF_subti_enrich['source'] = 'SubtiWiki'

DF_GO_enrich.rename({'gene_ontology':'annotation'},axis=1, inplace=True)
# DF_pathway_enrich.rename({'kegg_id':'annotation'},axis=1, inplace=True)
# DF_module_enrich.rename({'kegg_id':'annotation'},axis=1, inplace=True)
# DF_subti_enrich.rename({'value':'annotation'},axis=1, inplace=True)

DF_enrichments = pd.concat([DF_GO_enrich, DF_pathway_enrich, DF_module_enrich])

# Check for single gene iModulons

Some iModulons are dominated by a single, high-coefficient gene. These iModulons may result from:
1. Overdecomposition of the dataset to identify noisy genes
1. Artificial knock-out of single genes
1. Regulons with only one target gene

No matter what causes these iModulons, it is important to be aware of them. The find_single_gene_imodulons function identifies iModulons that are likely dominated by a single gene.

The iModulons identified by ``find_single_gene_imodulons`` may contain more than one gene, since a threshold-agnostic method is used to identify these iModulons.

In [30]:
sg_imods = ica_data.find_single_gene_imodulons(save=True)
len(sg_imods)

9

In [31]:
for i,mod in enumerate(sg_imods):
    ica_data.rename_imodulons({mod:'SG_'+str(i+1)})

# Save iModulon object

In [32]:
# Add iModulon sizes and explained variance
for im in ica_data.imodulon_names:
    ica_data.imodulon_table.loc[im,'imodulon_size'] = len(ica_data.view_imodulon(im))
    ica_data.imodulon_table.loc[im,'explained_variance'] = explained_variance(ica_data,imodulons=im)

This will save your iModulon table, your thresholds, and any other information stored in the ica_data object.

In [33]:
save_to_json(ica_data, os.path.join('..','data','cgu_raw.json.gz'))

If you prefer to view and edit your iModulon table in excel, save it as a CSV and reload the iModulon as before